# Epicrisis ONNX export (Colab, desde Drive)

Este cuaderno:
1) Monta Google Drive
2) Copia `epicrisis-merged.zip` desde Drive
3) Exporta a ONNX con Optimum
4) Reorganiza estructura para Transformers.js
5) Copia el ZIP final a Drive


In [ ]:
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Instalar dependencias (forzar versiones compatibles)
!pip -q uninstall -y optimum
!pip -q install "optimum==1.23.3" "transformers>=4.45" onnx onnxruntime


In [ ]:
# Verificar Optimum y exporters
import importlib.metadata as im
import importlib.util as iu
print('optimum:', im.version('optimum'))
print('exporters.onnx:', iu.find_spec('optimum.exporters.onnx') is not None)


In [ ]:
# Copiar el merged desde Drive
# Ajusta esta ruta si tu archivo esta en otra carpeta
MERGED_ZIP = '/content/drive/MyDrive/fine-tuning/epicrisis-merged.zip'
!cp "{MERGED_ZIP}" /content/epicrisis-merged.zip
!unzip -q /content/epicrisis-merged.zip -d /content
!ls -la /content/epicrisis-merged


## Exportar a ONNX (fp16 en GPU)
Si tu runtime tiene GPU CUDA, esto usara `device=cuda`.

In [ ]:
from optimum.exporters.onnx import main_export

main_export(
    model_name_or_path='/content/epicrisis-merged',
    output='/content/epicrisis-finetuned-onnx',
    task='text-generation-with-past',
    device='cuda',
    dtype='fp16',
    opset=18,
)


## Reorganizar estructura para Transformers.js (fp16)

In [ ]:
import os, shutil, json
out_dir = '/content/epicrisis-finetuned-onnx'
onnx_dir = os.path.join(out_dir, 'onnx')
os.makedirs(onnx_dir, exist_ok=True)

for f in os.listdir(out_dir):
    if f.endswith('.onnx') or f.endswith('.onnx_data'):
        shutil.move(os.path.join(out_dir, f), os.path.join(onnx_dir, f))

cfg_path = os.path.join(out_dir, 'config.json')
if os.path.exists(cfg_path):
    with open(cfg_path, 'r') as f:
        cfg = json.load(f)
    cfg['transformers.js_config'] = {
        'dtype': 'fp16',
        'kv_cache_dtype': {
            'fp16': 'float16'
        }
    }
    with open(cfg_path, 'w') as f:
        json.dump(cfg, f, indent=2)

print('Listo:', out_dir)


## Comprimir y copiar a Drive

In [ ]:
!zip -r /content/epicrisis-finetuned-onnx.zip /content/epicrisis-finetuned-onnx
!cp /content/epicrisis-finetuned-onnx.zip /content/drive/MyDrive/fine-tuning/
!ls -la /content/drive/MyDrive/fine-tuning/
